# EDA, Feature Engineering, and AutoML

In this lab, we will apply learned concepts from Day 3-4 lectures to perform data exploration, feature engineering, and autoML on a house sale dataset.

The goal of this assignment is to analyze 3 years’(2018-2020) house sales data provided by New York City (NYC) goverment and build regression model to predict house price. NYC has five boroughs, i.e., Bronx, Brooklyn, Manhattan, Queens and Staten Island. Sales of houses in each borough has been provided.


**Submission: submit via onq.** 


## Install auto-sklearn

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn==0.14.6

## Import Libiraries

In [ ]:
# Import needed libraries. E.g., pandas, missingno, and sklearn
import autosklearn
import pandas as pd
import numpy as np
import sklearn 
import matplotlib 
from matplotlib import pyplot as plt
import plotly.express as px
from google.colab import drive
drive.mount('/content/drive')
import missingno as msno

from scipy.stats import ttest_1samp,ttest_ind
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
from sklearn import preprocessing
from pprint import pprint

import sklearn.datasets
import sklearn.metrics

import autosklearn.regression
import matplotlib.pyplot as plt
import glob

from sklearn.metrics import mean_absolute_error
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.metrics import mean_absolute_error as auto_mean_absolute_error

## Task 1: 
* Read dataset
* Merge data  
* Perform basic data exploration

### Note: 
#### You should briefely discuss the quality of dataset (missing values, duplicate entries, etc.) Also, in task 3, you would be asked to perform prediction on house sale price, thus it would be good to consider outliers related to this prediciton task when you perform data cleaning.

In [ ]:
# read data and merge it

all_files = glob.glob("/content/drive/MyDrive/lab2-dataset_sha/*.csv")
data = pd.concat((pd.read_csv(f) for f in all_files))

In [ ]:
#check the shape of data
data.shape

In [ ]:
#read head of data
data.head(200)

In [ ]:
#sort data by BOROUGH
data.sort_values(by=['BOROUGH'], ascending=True,inplace=True)

In [ ]:
data

In [ ]:
#reset index
data=data.reset_index()
data=data.drop(columns=['index'])

In [ ]:
data

In [ ]:
#check data infro
data.info()

In [ ]:
#describe the data
data.describe()

In [ ]:
#check duplicate
data.duplicated().sum()

In [ ]:
#drop duplicates
data.drop_duplicates(inplace=True)

In [ ]:
#check nulls
data.isnull().sum()

In [ ]:
#check nulls percentage of each column
data.isnull().sum()* 100 / len(data)

In [ ]:
#drop rows that full of nulls
data.dropna(axis = 0, how = 'all', inplace = True)

In [ ]:
#check data nulls again
data.isnull().sum()

In [ ]:
msno.matrix(data)

In [ ]:
msno.heatmap(data)

### Depending on msno we need to drop (Easement and Apartment Number)

## Task 2
#### Data exploration using data visualization. Raise two questions that can be answered by performing data visualization. Briefely mention why you think this question would be interesting to whom (who is your audience). Think about the EDA principals.

### The question is too important for me, it will help me to select best features.

### Question 2.1
## Do the old builds with the same total units have higher price than new ones or not?

#### Data Exploration.

In [ ]:
#check nulls percentage of each column
data.isnull().sum()* 100 / len(data)

In [ ]:
#drop columns that contain very high missing data percentage
data.drop(columns=['EASE-MENT','APARTMENT NUMBER'],inplace=True)

In [ ]:
#check nulls percentage of each column
data.isnull().sum()* 100 / len(data)

In [ ]:
#check type of each column
data.dtypes

#### Firstly we need to explore and handle sale price

In [ ]:
# TODO write code to answer Q2.1
#check nulls in sale price column
np.count_nonzero(data['SALE PRICE'].isna())

In [ ]:
#check value counts on sale price
data['SALE PRICE'].value_counts()

In [ ]:
#remove , and $ from sale price data
data['SALE PRICE'] = data['SALE PRICE'].str.translate(str.maketrans({',':''}))
data['SALE PRICE'] = data['SALE PRICE'].str.translate(str.maketrans({'$':''}))

In [ ]:
#convert to numeric
data['SALE PRICE']=pd.to_numeric(data['SALE PRICE'])

In [ ]:
#check the ratio of missing data after cleaning
total_miss=data['SALE PRICE'].isnull().sum()
percent_miss = (total_miss/data['SALE PRICE'].isnull().count()*100) 
missing_data = {'Total missing':total_miss,'% missing':percent_miss}
missing_data

In [ ]:
data['SALE PRICE'].value_counts()

In [ ]:
#check unique values
data['SALE PRICE'].unique()

#### Secondly we need to handle number of units.

In [ ]:
#check nulls count in total units column
np.count_nonzero(data['TOTAL UNITS'].isna())

In [ ]:
#check nulls count in commercial units column
np.count_nonzero(data['COMMERCIAL UNITS'].isna())

In [ ]:
#check nulls count in residential units column
np.count_nonzero(data['RESIDENTIAL UNITS'].isna())

In [ ]:
#check count of each value
data['TOTAL UNITS'].value_counts()

In [ ]:
#check count od each value
data['RESIDENTIAL UNITS'].value_counts()

In [ ]:
#check unique values
data['RESIDENTIAL UNITS'].unique()

In [ ]:
#convert residential and commercial units to string
data['RESIDENTIAL UNITS']= data['RESIDENTIAL UNITS'].astype(str)
data['COMMERCIAL UNITS']= data['COMMERCIAL UNITS'].astype(str)

We need to replace nulls in RESIDENTIAL UNITS and COMMERCIAL UNITS with 0 value.

In [ ]:
#remove , from specific column
data['RESIDENTIAL UNITS'] = data['RESIDENTIAL UNITS'].str.translate(str.maketrans({',':''}))
data['COMMERCIAL UNITS'] = data['COMMERCIAL UNITS'].str.translate(str.maketrans({',':''}))

In [ ]:
#check unique values
data['RESIDENTIAL UNITS'].unique()

In [ ]:
#fill nulls with 0 and convert to numeric
data['RESIDENTIAL UNITS'] = pd.to_numeric(data['RESIDENTIAL UNITS'], errors='coerce').fillna(0, downcast='infer')
data['COMMERCIAL UNITS'] = pd.to_numeric(data['COMMERCIAL UNITS'], errors='coerce').fillna(0, downcast='infer')

In [ ]:
data['RESIDENTIAL UNITS'].value_counts()

In [ ]:
#check nulls after cleaning
print(np.count_nonzero(data['RESIDENTIAL UNITS'].isna()))
print(np.count_nonzero(data['COMMERCIAL UNITS'].isna()))

In [ ]:
data['COMMERCIAL UNITS'].unique()

In [ ]:
data['RESIDENTIAL UNITS'].unique()

Handle nulls in TOTAL UNITS by filling it by the sum of RESIDENTIAL UNITS and COMMERCIAL UNITS.

In [ ]:
#fill total units nulls
data['TOTAL UNITS']=(data['RESIDENTIAL UNITS']+data['COMMERCIAL UNITS'])

In [ ]:
data['TOTAL UNITS'].unique()

In [ ]:
#check the ratio of missing data in TOTAL UNITS after cleaning
total_miss=data['TOTAL UNITS'].isnull().sum()
percent_miss = (total_miss/data['TOTAL UNITS'].isnull().count()*100) 
missing_data = {'Total missing':total_miss,'% missing':percent_miss}
missing_data

In [ ]:
data['TOTAL UNITS'].value_counts()

We will use TOTAL UNITS value = 1 to compare between build year because it is most frequent one.

#### Then we need to explore and handle year build.

In [ ]:
data['YEAR BUILT'].unique()

As we see Year Build have 0 value should be replaced with nulls to handel it after that.

In [ ]:
#replace 0 with nulls to handle it with imputer at task 4
data['YEAR BUILT'].replace((0.), np.nan, inplace=True)
data['YEAR BUILT'].replace((0), np.nan, inplace=True)
data['YEAR BUILT'].replace(('0'), np.nan, inplace=True)

In [ ]:
data['YEAR BUILT'].unique()

In [ ]:
#check sale price of each year with total unit
sns.regplot(x='YEAR BUILT', y='SALE PRICE',
            data=data[data['YEAR BUILT']<1800][data['TOTAL UNITS']<=5],
            fit_reg=False, scatter_kws={'alpha':1})

As we see any year before 1800 categorized as outlier, so we can drop them after that.

In [ ]:
#check sale price at year build after 1800
sns.regplot(x='YEAR BUILT', y='SALE PRICE',
            data=data[data['YEAR BUILT']>1800][data['TOTAL UNITS']<=5],
            fit_reg=False, scatter_kws={'alpha':1})

In [ ]:
#check after 1900
sns.regplot(x='YEAR BUILT', y='SALE PRICE',
            data=data[data['YEAR BUILT']>=1900][data['TOTAL UNITS']<=5],
            fit_reg=False, scatter_kws={'alpha':1})

As we see the prices of 90s buildings is greater than 20s. But we can visualize each no. of units alone to make sure about that.

We will check for no. of units 1 because it the most frequent value.

In [ ]:
#check when the total unit only =1
sns.regplot(x='YEAR BUILT', y='SALE PRICE',
            data=data[data['YEAR BUILT']>=1900][data['TOTAL UNITS']==1],
            fit_reg=False, scatter_kws={'alpha':1})

As we see in previous result, 90s prices is greater than 20s, specially the range from 1920 till 1970 and the prices of 2015 too.

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='TOTAL UNITS', y='SALE PRICE', data=data)
plt.title('TOTAL UNITS vs SALE PRICE')
plt.show()

### Answer to Question 1
## Yes, old buildings have higher price than new ones with the same total units.

### Question 2.2
## Does zip code indicate to the same Neighborhood?

#### Handeling values of borough column.

In [ ]:
# TODO write code to answer Q2.2
data['BOROUGH'].value_counts()

In [ ]:
data['BOROUGH'] = data['BOROUGH'].replace(1.0, "Manhattan")
data['BOROUGH'] = data['BOROUGH'].replace(2.0, "Bronx")
data['BOROUGH'] = data['BOROUGH'].replace(3.0, "Brooklyn")
data['BOROUGH'] = data['BOROUGH'].replace(4.0, "Queens")
data['BOROUGH'] = data['BOROUGH'].replace(5.0, "Staten Island")

In [ ]:
data['BOROUGH'] = data['BOROUGH'].astype(str)
data['BOROUGH']=data['BOROUGH'].astype('category')

In [ ]:
data['BOROUGH'].value_counts()

#### Now we will handle zip code values

In [ ]:
#describe the data
data.describe()

Zip Code contains 0 values should be replaced with nulls to handle it.

In [ ]:
#replace 0 with nulls
data['ZIP CODE']= data['ZIP CODE'].replace(0.,np.nan)

In [ ]:
#check nulls percentage of each column
data.isnull().sum()* 100 / len(data)

We can drop Nulls from columns TAX CLASS AT PRESENT, BUILDING CLASS AT PRESENT and ZIP CODE


In [ ]:
#drop useless columns
data=data.dropna(subset=['TAX CLASS AT PRESENT', 'BUILDING CLASS AT PRESENT','ZIP CODE'])

In [ ]:
data['ZIP CODE'].unique()

In [ ]:
data['ZIP CODE'].value_counts()

#### We'll visualize our data to answer the question.

In [ ]:
data['NEIGHBORHOOD'].value_counts()

In [ ]:
#visualize specific parts of data
plt.figure(figsize=(15,6))
sns.boxplot(x='NEIGHBORHOOD', y="ZIP CODE", data=pd.concat([data[data['NEIGHBORHOOD']=='FLUSHING-NORTH'],
                                                            data[data['NEIGHBORHOOD']=='UPPER EAST SIDE (59-79)'],
                                                            data[data['NEIGHBORHOOD']=='UPPER WEST SIDE (59-79)'],
                                                            data[data['NEIGHBORHOOD']=='UPPER EAST SIDE (79-96)'],
                                                            data[data['NEIGHBORHOOD']=='BEDFORD STUYVESANT'],
                                                            data[data['NEIGHBORHOOD']=='MORNINGSIDE HEIGHTS']
                                                            ],axis=0))

In [ ]:
sns.catplot(x="NEIGHBORHOOD", y="ZIP CODE", hue="BOROUGH", kind="bar",height=7, 
            data=data[data['BOROUGH']=='Bronx'])
plt.xticks(rotation=90)

In [ ]:
sns.catplot(x="NEIGHBORHOOD", y="ZIP CODE", hue="BOROUGH",height=6, kind="bar", data=pd.concat([data[data['NEIGHBORHOOD']=='FLUSHING-NORTH'],
                                                                                                data[data['NEIGHBORHOOD']=='UPPER EAST SIDE (59-79)'],
                                                                                                data[data['NEIGHBORHOOD']=='UPPER WEST SIDE (59-79)'],
                                                                                                data[data['NEIGHBORHOOD']=='UPPER EAST SIDE (79-96)'],
                                                                                                data[data['NEIGHBORHOOD']=='BEDFORD STUYVESANT'],
                                                                                                data[data['NEIGHBORHOOD']=='MORNINGSIDE HEIGHTS']
                                                                                                ],axis=0))
plt.xticks(rotation=90)

In [ ]:
sns.catplot(x="NEIGHBORHOOD", y="ZIP CODE", hue="BOROUGH", kind="bar",height=5, 
            data=data[data['BOROUGH']=='Queens'][data['NEIGHBORHOOD']=='FLUSHING-NORTH'])

In [ ]:
sns.catplot(x="NEIGHBORHOOD", y="ZIP CODE", hue="BOROUGH", kind="bar",height=5, data=data[data['NEIGHBORHOOD']=='UPPER EAST SIDE (59-79)'])

### Answer to Question 2
## The Neighborhood at specific borough contains the same zipcode.

### Exploring data

In [ ]:
# house prices greater than 5 mln probably represents outliers.
import matplotlib.ticker as ticker

sns.set_style("whitegrid")
plt.figure(figsize=(10,5))
plotd = sns.distplot(data[(data['SALE PRICE']>100) & (data['SALE PRICE'] < 5000000)]['SALE PRICE'], kde=True, bins=100)

tick_spacing=250000 # set spacing for each tick
plotd.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plotd.set_xlim([-100000, 5000000]) # do not show negative values 
plt.xticks(rotation=30) # rotate x ticks by 30 degrees
plt.axvline(data[(data['SALE PRICE']>100) & (data['SALE PRICE'] < 5000000)]['SALE PRICE'].mean(), c='red')
plt.axvline(data[(data['SALE PRICE']>100) & (data['SALE PRICE'] < 5000000)]['SALE PRICE'].median(), c='blue')
plt.text(250000,0.0000012, "median")
plt.text(850000,0.0000010, "mean")
plt.show()

In [ ]:
df = data[(data['SALE PRICE'] > 10000) & (data['SALE PRICE'] < 10000000)]

plt.figure(figsize=(12,5))
sns.displot(df['SALE PRICE'], bins=40, rug=True)
plt.show()

## Task 3
#### Data Exploration via Statistical Test Raise one question that can be answered by performing hypothesis test. Briefely mention why you think this question would be interesting to whom (who is your audience). Also mention which statistical test you would choose and why.

### Null hypothesis: {BUILDING CLASS has the same sale price at sale and at present}

### Alternative hypothesis: {BUILDING CLASS doesn't have the same sale price at sale and at present}

In [ ]:
# TODO write code to perform task 3
#check value counts in this column
data['BUILDING CLASS AT PRESENT'].value_counts()

In [ ]:
#check value counts
data['BUILDING CLASS AT TIME OF SALE'].value_counts()

In [ ]:
#convert features to category
data['BUILDING CLASS AT TIME OF SALE']=data['BUILDING CLASS AT TIME OF SALE'].astype('category')
data['BUILDING CLASS AT PRESENT']=data['BUILDING CLASS AT PRESENT'].astype('category')

In [ ]:
#check null hypothies with statistics
from scipy import stats
sale = data[data['BUILDING CLASS AT TIME OF SALE'] == 'D4']['SALE PRICE']
present = data[data['BUILDING CLASS AT PRESENT'] == 'D4']['SALE PRICE']
stats.ttest_ind(sale, present)

## pvalue < 0.05, it means Alternative hypothies is right.

## Task 4
#### Feature Engineering. If we would like to predict the house sale price. Analyze the scale of each attribute and determine which ones you would transfer (e.g., cateogorical features). Discuss how you plan to select important features.

* We can drop Neighborhood column and Address because we have zipcode which refers to location.
* Convert date to month and year.
* Drop building class category because it's the same with building calss at present.

In [ ]:
# TODO for Task 4
# 'SALE DATE' can be split into month and year

data['SALE DATE']= pd.to_datetime(data['SALE DATE'], errors='coerce')
data['SALE MONTH'] = pd.DatetimeIndex(data['SALE DATE']).month.astype("category")
data['SALE YEAR'] = pd.DatetimeIndex(data['SALE DATE']).year.astype("category")

In [ ]:
data=data.drop(columns=['ADDRESS','NEIGHBORHOOD','SALE DATE','BUILDING CLASS CATEGORY'])

In [ ]:
data['LAND SQUARE FEET'].value_counts()

In [ ]:
data['LAND SQUARE FEET'].unique()

In [ ]:
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].str.translate(str.maketrans({',':''}))
data['LAND SQUARE FEET'] = data['LAND SQUARE FEET'].str.translate(str.maketrans({',':''}))

In [ ]:
data['LAND SQUARE FEET'].unique()

In [ ]:
msno.heatmap(data)

As we see, land square feet and gross square feet are highly corelated missing.


In [ ]:
data.isnull().sum()* 100 / len(data)

In [ ]:
#REPLACE 0 OF SALE PRICE WITH NULLS TO FILL IT BY IMPUTER
data['SALE PRICE'].replace((0), np.nan, inplace=True)

In [ ]:
#fill nulls
from sklearn.impute import KNNImputer
imputed = ['LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT','SALE PRICE']
imputer = KNNImputer(n_neighbors=2)
data[imputed] = imputer.fit_transform(data[imputed])

In [ ]:
#check nulls
data.isnull().sum()* 100 / len(data)

In [ ]:
#encode categorical data
from sklearn.preprocessing import LabelEncoder
data['BOROUGH'] = LabelEncoder().fit_transform(data['BOROUGH'])
data['BUILDING CLASS AT PRESENT'] = LabelEncoder().fit_transform(data['BUILDING CLASS AT PRESENT'])
data['BUILDING CLASS AT TIME OF SALE'] = LabelEncoder().fit_transform(data['BUILDING CLASS AT TIME OF SALE'])
data['TAX CLASS AT PRESENT'] = LabelEncoder().fit_transform(data['TAX CLASS AT PRESENT'])
data['TAX CLASS AT TIME OF SALE'] = LabelEncoder().fit_transform(data['TAX CLASS AT TIME OF SALE'])

In [ ]:
#display data after encoding
data

In [ ]:
#convert some of features to category
categ = ['BOROUGH','LOT','BLOCK']
for col in categ:
    data[col] = data[col].astype('category')

In [ ]:
#check type of each column
data.dtypes

In [ ]:
#split data and label
x=data.drop(columns=['SALE PRICE'])
y = data.iloc[:,-1].values.reshape(-1,1)

In [ ]:
#check shapes
y.shape,x.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
#rescale train data to be simpler
sc = StandardScaler()
x= sc.fit_transform(x)
# y_= sc.fit_transform(y)

## Task 5: AutoML

#### Using Auto-sklearn to explore performance of one state-of-the-art autoML tool on the given data (after your previous preprocessing). Optional: compare with performing autosklearn on raw data.
#### ref. https://automl.github.io/auto-sklearn/master/examples/20_basic/example_regression.html

In [ ]:
# TODO for Task 5
#split data to train and test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y,test_size=0.2, random_state=1)

In [ ]:
#use automl regression model
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
)
automl.fit(X_train, y_train, dataset_name='diabetes')
print(automl.leaderboard())

In [ ]:
#predict train and test data then calculate r2 score/ mae
train_predictions = automl.predict(X_train)
automl.score(X_train,y_train)
automl.score(X_test,y_test)

print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))
print("Test MAE:", mean_absolute_error(y_test, test_predictions))